##  MiniFlow

### Introduction 

A neural network is a graph of mathematical funstions such as linear combinations and activation functions. The graph consists of nodes, and edges.

Nodes in each layer(except for the input layer) perform mathematical functions using inputs from nodes in the previous layers. For example, a node could represent $f(x,y) = x + y$, where $x$ and $y$ are input values from nodes in the previous layer.

The edges in the graph describe the connection between the nodes, along which the values flow from one layer to the next.

MiniFlow is a neural network library meant to be a miniature version of Google's TensorFlow library. The library implements backpropogation and forward passing using a simple sigmoid activation function. 


In [58]:
import numpy as np

### The Node class

In [59]:
class Node():
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        
        # For each inbound Node here, add this Node as an outbound Node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # A calculated value
        self.value = None
        
    def forward(self):
        """
        Forword propagation.
        
        Compute the output value based on 'inbound_nodes' and 
        store the result in self.value.
        """
        raise NotImplemented
        


### Input node class

Input nodes do no calculations but only hold values to be passed forward

In [60]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes
        Node.__init__(self)
        
    def forward(self, value=None):
        # Overwrite the value if one is passed in
        if value is not None:
            self.value = value
            

### Add node class

Takes 2 inbound nodes, x, and y, and adds the values of those nodes.

In [61]:
class Add(Node):
    def __init__(self, *args):
        Node.__init__(self, args)
    
    def forward(self):
        """
        Calculates sum and passes it forward
        """
        self.value = sum([node.value for node in self.inbound_nodes])

### Function to run a topological sort on a list of nodes

This function performs a topological sort based on Khan's algorithm.

In [62]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

### Forward pass

Perform forward pass on sorted nodes

In [63]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

### Test addition with varying number of input nodes

In [64]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 + 5 = 15 (according to miniflow)


In [65]:
"""
No need to change anything here!

If all goes well, this should work after you
modify the Add class in miniflow.py.
"""

x, y, z = Input(), Input(), Input()

f = Add(x, y, z)

feed_dict = {x: 4, y: 5, z: 10}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

# should output 19
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))


4 + 5 + 10 = 19 (according to miniflow)


### Multiplication node class

In [66]:
class Mul(Node):
    def __init__(self, *args):
        Node.__init__(self, args)
    
    def forward(self):
        """
        Calculates product and passes it forward
        """
        self.value = 1
        for node in self.inbound_nodes:
            self.value*=node.value 

###  Test multiplication with varying number of nodes

In [67]:
x, y = Input(), Input()

f = Mul(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} * {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 * 5 = 50 (according to miniflow)


In [68]:
x, y, z = Input(), Input(), Input()

f = Mul(x, y, z)

feed_dict = {x: 4, y: 5, z: 10}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

# should output 200
print("{} * {} * {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))


4 * 5 * 10 = 200 (according to miniflow)


Consider the following equation

$$\sigma = \sum_{i} x_iw_i + b$$

inputs, x (vector)  
weights, w (vector)  
bias, b (scalar)

See below for a node/neuron that applies the above linear equation

### Linear Node class

In [69]:
class Linear(Node):
    def __init__(self, X, W, b):
        Node.__init__(self, [X, W, b])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.
        """
        from numpy import dot
        X = self.inbound_nodes[0].value # inputs
        W = self.inbound_nodes[1].value # weights
        assert len(X) == len(W),"Inconsistent number of inputs and weights"
        b = self.inbound_nodes[2].value # bias
        self.value = dot(X, W) + b # Z = XW + b

### Test Linear node with matrices

In [70]:
import numpy as np

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)


[[-9.  4.]
 [-9.  4.]]


### Sigmoid node

The sigmoid function is defined as follows

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

The sigmoid node will take one input and apply the above function during a forward pass, passing any result to the next layer, if there is one

In [71]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))
    
    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

In [72]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


### Calculating loss/cost

We need a way to determine how correct the output of our network is, we can calculate this using the mean squared error (MSE). Like so:

$$C(w,b) = \frac{1}{m}\sum_{x}\vert\vert y(x) - a\vert\vert^2$$

We shall implement this in the forward pass of it's own node

In [80]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        m = self.inbound_nodes[0].value.shape[0]
        self.value = np.mean(np.square(y-a))

### Test MSE

In [81]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(cost, graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667
